Imports

In [ ]:
import json
from math import floor
from random import shuffle
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from numpy import ravel
from sklearn.metrics import accuracy_score

Filters `original_dataset.json` and removes everything that doesn't `cs.` in it. After that, creates new file

In [ ]:
data = []
with open('original_dataset.json', 'r') as f:
    for line in f:
        data.append(json.loads(line))
filtered = [x for x in data if (' cs.' in x['categories'] or x['categories'][:3] == 'cs.' )]

with open('csDataset.json', 'w') as f:
    f.write(json.dumps(filtered, indent=2))

Splits dataset into two random parts. 80% for training and 20% for testing

In [ ]:
with open('csDataset.json', 'r') as f:
    data = json.loads(f.read())
shuffle(data)
length = len(data)
train_data_size = floor(length * 80/100)
train_data = data[train_data_size:]

with open('trainDataset.json', 'w') as f:
    f.write(json.dumps(data[:train_data_size], indent=2))

with open('testDataset.json', 'w') as f:
    f.write(json.dumps(data[train_data_size:], indent=2))

Creating topics array and dictionary

In [ ]:
d = []
d.append("cs.AI_Artificial Intelligence")
    
d.append("cs.CL_Computation and Language")
    
d.append("cs.CC_Computational Complexity")
    
d.append("cs.CE_Computational Engineering, Finance, and Science")
    
d.append("cs.CG_Computational Geometry")
    
d.append("cs.GT_Computer Science and Game Theory")
    
d.append("cs.CV_Computer Vision and Pattern Recognition")
    
d.append("cs.CY_Computers and Society")

d.append("cs.CR_Cryptography and Security")
    
d.append("cs.DS_Data Structures and Algorithms")
    
d.append("cs.DB_Databases")
    
d.append("cs.DL_Digital Libraries")
    
d.append("cs.DM_Discrete Mathematics")
    
d.append("cs.DC_Distributed, Parallel, and Cluster Computing")
    
d.append("cs.ET_Emerging Technologies")
    
d.append("cs.FL_Formal Languages and Automata Theory")

d.append("cs.GL_General Literature")

d.append("cs.GR_Graphics")

d.append("cs.AR_Hardware Architecture")

d.append("cs.HC_Human-Computer Interaction")

d.append("cs.IR_Information Retrieval")

d.append("cs.IT_Information Theory")

d.append("cs.LO_Logic in Computer Science")

d.append("cs.LG_Machine Learning")

d.append("cs.MS_Mathematical Software")

d.append("cs.MA_Multiagent Systems")

d.append("cs.MM_Multimedia")

d.append("cs.NI_Networking and Internet Architecture")

d.append("cs.NE_Neural and Evolutionary Computing")

d.append("cs.NA_Numerical Analysis")

d.append("cs.OS_Operating Systems")

d.append("cs.OH_Other Computer Science")

d.append("cs.PF_Performance")

d.append("cs.PL_Programming Languages")

d.append("cs.RO_Robotics")

d.append("cs.SI_Social and Information Networks")

d.append("cs.SE_Software Engineering")

d.append("cs.SD_Sound")

d.append("cs.SC_Symbolic Computation")

d.append("cs.SY_Systems and Control")

categoriesDict = {x.split('_')[0] : x.split('_')[1] for x in d}

categories = [x.split('_')[0] for x in d]
print(len(categories))

Imports `trainJson` and `testJson` files into variables

In [ ]:
with open('trainDataset.json', 'r') as f:
    trainJson = pd.read_json(f.read())

with open('testDataset.json', 'r') as f:
    testJson = pd.read_json(f.read())


Cleans and takes keywords from the abstract of articles

In [ ]:
def cleanText(text):
    review = re.sub(r'\$.*?\$', '', text)
    review = re.sub('[^a-zA-Z-]', ' ', review)
    review = review.lower()
    review = review.split()
    stops = stopwords.words("english")
    ps = WordNetLemmatizer()
    review = [ps.lemmatize(word) for word in review if not word in set(stops)]
    review = ' '.join(review)
    return review

def cleaning(trainSet):
    corpus = []
    for ind, i in trainSet.iterrows():
        corpus.append(cleanText(i['title'] + " " + i['abstract']))
    return corpus


corpus = cleaning(trainJson)
corpus_test = cleaning(testJson)

Create vectors depending on words corpus for each document

In [ ]:
cv = TfidfVectorizer(ngram_range=(1,3),max_df=0.45, max_features=70, stop_words='english', min_df=2)
X = cv.fit_transform(corpus).toarray()


cv_test = TfidfVectorizer(ngram_range=(1,3), max_df=0.45, max_features=70, stop_words='english', min_df=2)
X_test = cv_test.fit_transform(corpus_test).toarray()


`Y` Dataset creation for classlabel

In [ ]:
def createY(datas):
    y = []
    for ind, i in datas.iterrows():
        ii = i['categories'].split()
        k = False
        for j in ii:
            if(j[:3] == "cs." and k == False):
                y.append(categories.index(j))
                k=True
    return y
y = pd.DataFrame(createY(trainJson))
y_test = pd.DataFrame(createY(testJson))

$\text{KNeighbors Classification Model}$

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

cat = KNeighborsClassifier(n_neighbors=80)

cat.fit(X, ravel(y))
y_pred = cat.predict(X_test)

accuracy_score(y_test, y_pred)

Finds Closest Text that is in the dataset

In [13]:
from sklearn.neighbors import NearestNeighbors
from math import log

text = """
In machine learning, support vector machines (SVMs, also support vector networks[1]) are supervised learning models with associated learning algorithms that analyze data for
classification and regression analysis. Developed at AT&T Bell Laboratories by Vladimir Vapnik with colleagues (Boser et al., 1992, Guyon et al., 1993, Cortes and Vapnik, 1995,
[1] Vapnik et al., 1997[citation needed]) SVMs are one of the most robust prediction methods, being based on statistical learning frameworks or VC theory proposed by Vapnik (1982, 1995) 
and Chervonenkis (1974). Given a set of training examples, each marked as belonging to one of two categories, an SVM training algorithm builds a model that assigns new examples to one category
or the other, making it a non-probabilistic binary linear classifier (although methods such as Platt scaling exist to use SVM in a probabilistic classification setting). SVM maps training examples
to points in space so as to maximise the width of the gap between the two categories. New examples are then mapped into that same space and predicted to belong to a category based on which side of the gap they fall.
 """

def tfCalculator(arr):
    freq={}
    
    for word in arr:
        if(word in freq):
            freq[word] += 1
        else:
            freq[word] = 1
    fet_arr = cv.get_feature_names_out()

    lena = 1
    for i in fet_arr:
        if i in arr:
            lena+=1

    for word in freq:
        freq[word] /= lena
    return freq

def newTFIDFArray(review):
    arr = review.split()

    freq=tfCalculator(arr)
    fet_arr = cv.get_feature_names_out()
    new_TFIDF = [0]*len(fet_arr)
    
    for word in arr: 
        jima = 1
        if(word in fet_arr):
            index = np.where(fet_arr == word)[0][0]
            
            for document in X:
                if document[index] != 0:
                    jima+=1
            
            idf = log(len(X)/jima,2)
            tfidf = freq[word] * idf
            new_TFIDF[index] = tfidf

    
    return np.array([new_TFIDF])


def kneigh(text, n):
    print('\n')
    review = cleanText(text)

    shemogzavnili = newTFIDFArray(review)
    
    memezoble = NearestNeighbors(n_neighbors=20).fit(np.append(X, shemogzavnili, axis=0))

    answer = memezoble.kneighbors(shemogzavnili, 20, return_distance=False)
    predCat = categoriesDict[categories[cat.predict(shemogzavnili)[0]]]
    print('\n'+"The text is about " + predCat)
    print("related articles abstracts:\n")
    i = 1
    while(i < n+1):
        if(trainJson.iloc[answer[0][i]]['license'] == None):
            continue
        if (int(trainJson.iloc[answer[0][i]]['update_date'].split('-')[0]) < 2015):
            print("Warning, This article may be outdated ("+trainJson.iloc[answer[0][i]]['update_date'].split('-')[0]+")\n")
        print(trainJson.iloc[answer[0][i]]['title'] +'\n'+ trainJson.iloc[answer[0][i]]['abstract'])
        print("   To view full article, use following link: " + "https://arxiv.org/pdf/"+trainJson.iloc[answer[0][i]]['id'])
        print('\n')
        i+=1
        
    

kneigh(text, 3)





The text is about Machine Learning
related articles abstracts:

Variational Learning with Disentanglement-PyTorch
  Unsupervised learning of disentangled representations is an open problem in
machine learning. The Disentanglement-PyTorch library is developed to
facilitate research, implementation, and testing of new variational algorithms.
In this modular library, neural architectures, dimensionality of the latent
space, and the training algorithms are fully decoupled, allowing for
independent and consistent experiments across variational methods. The library
handles the training scheduling, logging, and visualizations of reconstructions
and latent space traversals. It also evaluates the encodings based on various
disentanglement metrics. The library, so far, includes implementations of the
following unsupervised algorithms VAE, Beta-VAE, Factor-VAE, DIP-I-VAE,
DIP-II-VAE, Info-VAE, and Beta-TCVAE, as well as conditional approaches such as
CVAE and IFCVAE. The library is compatible 